In [ ]:
# Global import -- to be run with any following cell
import glob
import pdb
import cProfile
import os
import networkx as nx
from lxml import etree
from graphs import call_graph
from graphs.devrank import devrank
from graphs.pagerank import pagerank
from graphs.call_commit_graph import CallCommitGraph

In [ ]:
# Call graph -- to be run with any following cell
# Input
xml_dir = '~/Persper/repos/linux-xml/kernel' # v4.10

xml_dir = os.path.expanduser(xml_dir)
c_roots = []
for xml in glob.glob(xml_dir + '/**/*.c.xml', recursive=True):
    tree = etree.parse(xml)
    c_roots.append(tree.getroot())
G, func_to_file = call_graph.c.build_call_graph(c_roots, {})
print("Number of nodes: {}".format(len(G.nodes())))
print("Number of edges: {}".format(len(G.edges())))
print("Number of connected components: {}".format(nx.number_weakly_connected_components(G)))

In [ ]:
# Print functions -- to be run with following cells

def rel_path(path):
    pre = os.path.commonprefix((xml_dir, '/' + path))
    # TODO: Append '/' to all func_to_file items. Use a standard form.
    return path[len(pre):]

def to_csv(index, item):
    # Potentially, this format can be output and used by Excel
    file = rel_path(func_to_file[item[0]]) if item[0] in func_to_file else 'Unknown'
    return str(index) + ', ' + item[0] + ', ' + str(item[1]) + ',' + file

def print_top_bottom(ranks, n):
    sorted_ranks = sorted(ranks.items(), key=lambda r: r[1])
    
    top = [r for r in reversed(sorted_ranks[-n:])]
    bottom = [r for r in sorted_ranks[:n]]

    print('Top ' + str(n))
    for i, r in enumerate(top):
        print(to_csv(i + 1, r))
        
    print('Bottom ' + str(n))
    for i, r in enumerate(bottom):
        print(to_csv(i + 1, r))

In [ ]:
# PageRank with a specific alpha
# Input
alpha = 0.5
num_func = 20 # top/bottom n to list

pr = pagerank(G, alpha=alpha)
print_top_bottom(pr, num_func)

In [ ]:
# DevRank with a specific alpha
# Input
alpha = 0.5
num_func = 20 # top/bottom n to list

dr = devrank(G, alpha=alpha)
print_top_bottom(dr, num_func)

In [ ]:
# DevRank with a range of alpha
# Input
alpha_low = 0.1
alpha_high = 0.9
alpha_step = 0.1
num_func = 20 # top/bottom n to list

a = alpha_low
while (a <= alpha_high):
    print("alpha = " + str(a))
    dr = devrank(G, alpha=a)
    print_top_bottom(dr, num_func)
    a += alpha_step

In [ ]:
# PageRank with a range of alpha
# Input
alpha_low = 0.1
alpha_high = 0.9
alpha_step = 0.1
num_func = 20 # top/bottom n to list

a = alpha_low
while (a <= alpha_high):
    print("alpha = " + str(a))
    dr = pagerank(G, alpha=a)
    print_top_bottom(dr, num_func)
    a += alpha_step

In [ ]:
# Call-commit graph

g = CallCommitGraph('~/Persper/repos/linux')
#g.process(from_beginning=True, num_commits=5)
g.process(rev='v4.10', num_commits=5)
g.devrank_functions(0.85)